# Study In Recommendation Engine

## Abstract

A recommender system, or a recommendation engine intends to seek prediction of 'rating' or 'preference' for a user that would rank an item or event, mainly used in commercial web applications. 

They have been utilized in various areas over the last ten years, commonly used as playlist generators for video and music services such as Youtube, Netflix, and BiliBili, product recommenders like Amazon, Trademe, and Ebay, and content or news based recommenders for social media platforms as Facebook, Twitter and Instergram. Also , there are other popular recommender systems being utilized for specific topics as hotels booking , dating matching ,and online competition game team up.

Flow and monetization are the core concept assocaited with commercial web applications for internet. Simply speaking , flow is the measurement that the number of visitings for a web application, while monetization evaluates its overall income for keeping the business up and running continuously and healthly. ( i.e. advertisement income of Youtube, membership subscriptions for Netflix, and fees/charges for each successfuly sale for Amazon ). 

Therefore, recommendation engine is one of the key part for monetization, that allows finding consumers' real demand through directing them to their most interest items or services. In addtion to this , owners of  commercial web applications, would be able accurately deploy advertisements and services to their customers based on a successful recommendation engine set up, for example ,playlist generators (youtube) could generate just right adverstisement to their viewers , and product recommenders like Amazon could pushed customers related products when they are viewing a certain item. Thus, it would optimise the resource usage and amplify income for a commercial organisation who has developed such kind of commercial web applications. 

In this project , we would implement a 'breath first' strategy so as to explore as much lifecycle and implementations for a recommendation engine , such as the input and output data structure, machine learning algorithms, evaluation standard for comparison of different algorithms, and its position and role in arhciteture of a commercial web applications , instead of undertaking in depth research on specific algorithms as per their theory and implementations, we will simply use an existing library to compute our findings and recommendations.






## Indroduction


## Data Collection

## EDA

## Algorithm Research

## Presentation of Web App

## Recommendation Engine in System Architecture 